# Tagoloan Cleaner
The purpose of this notebook is to clean the sensor value of Tagoloan's Rain Gauge given First Gen's sensor specification where the sensor value maxes out at 819.2mm

## Import the necessary libraries

Note: 
1. ***Rain Gauge*** is the only one need of cleaning. <u>Water Level</u> is already accurate
2. ***Rain Gauge 2*** is already Accurate.

In [5]:
import os
from pathlib import Path
import pandas as pd

rg1_raw     = Path(r"Rain Gauge\Tagoloan\Rain Gauge 1\Raw")
rg2_raw     = Path(r"Rain Gauge\Tagoloan\Rain Gauge 2\Raw")
final_rg1   = Path(r"Rain Gauge\Tagoloan\Rain Gauge 1")
final_rg2   = Path(r"Rain Gauge\Tagoloan\Rain Gauge 2")


### Add all necessary functions

In [6]:
def return_all_files(directory,extension):
    """Returns all files found in the directory given the extension"""
    all_files = []

    for filename in os.listdir(directory):
        f = os.path.join(directory,filename)
        if os.path.isfile(f):
            if f.split('.')[-1] == extension:
                all_files.append(Path(f))
    return all_files

def rg_correction(df):
    """Returns corrected Sensor Value as a list"""
    
    df['New Timestamp'] = df['Timestamp'].apply(lambda x: pd.to_datetime(x))
    max_val = 819.2
    sum_counter = []
    cor_val_list = [df.loc[0, 'Sensor Value']]
    for idx in range(1,df.shape[0]):
        cur_val = df.loc[idx, 'Sensor Value']
        prev_val = df.loc[idx-1, 'Sensor Value']
        cur_time = df.loc[idx, 'New Timestamp']
        prev_time = df.loc[idx-1, 'New Timestamp']

        hours_diff = (cur_time - prev_time).seconds / 3600
        #First catches if sensor dips, second catches if the time difference is just 1 hour
        if ((cur_val - prev_val) < 0 and hours_diff <= 1) and (max_val-prev_val+cur_val < max_val*0.1) :
            cor_val = max_val-prev_val+cur_val
            new_cor_val = cor_val+prev_val
            cor_val_list.append(new_cor_val)
            sum_counter.append(new_cor_val)
            continue
        cor_val = cur_val + sum(sum_counter)
        cor_val_list.append(cor_val)
    
    df.drop(columns=["New Timestamp"], inplace=True)
    return cor_val_list



### Main Loop

In [7]:
import threading

def cleaner(inp,out):
    directory = inp
    xl_files = return_all_files(directory, 'xlsx')

    for file in xl_files:
        df = pd.read_excel(file)
        df['Sensor Value'] = rg_correction(df)
        head, tail = os.path.split(file)
        tail = 'Processed_'+tail
        df.to_excel(os.path.join(out,tail))
def transfer(inp,out):
    directory = inp
    xl_files = return_all_files(directory, 'xlsx')

    for file in xl_files:
        df = pd.read_excel(file)
        head, tail = os.path.split(file)
        tail = 'Processed_'+tail
        df.to_excel(os.path.join(out,tail))

print("Starting process...")
threads = []
#RG1
for inp, out in zip([rg1_raw], [final_rg1]):
    t = threading.Thread(target=cleaner, args=[inp,out])
    t.start()
    threads.append(t)
#RG2
for inp, out in zip([rg2_raw], [final_rg2]):
    t = threading.Thread(target=transfer, args=[inp,out])
    t.start()
    threads.append(t)

for t in threads:
    t.join()
print("Finished cleaning")
    # count += 1
    # perc = count/len(all_rg_path_input)*100
    # print("Process is {:.2f} percent complete".format(perc))    


Starting process...


c:\Users\sulay\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


c:\Users\sulay\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\sulay\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\sulay\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\sulay\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\sulay\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains

Finished cleaning
